# _ANALYSIS OF MULTIPLE LINEAR REGRESSION_

------------------------------------------------------------------------

#### ___Made by (Kelompok 1):___<br>5003221061 - M. Iqbal Nurrifki<br>5003221120 - Rasendra Akbar S<br>5003221131 - Ketut Bagus Ananta<br>5003221140 - M. Noor Satriya<br>5003221151 - Yansen Demos Putra N

#### Link Data yang digunakan : __https://drive.google.com/drive/folders/159dRUHZC3h8aGNmkPsbR4g1CdglU7wHJ?usp=sharing__

--------------------------------------------------------------------------

In [23]:
#pip install scipy

In [24]:
#pip install statsmodels

In [25]:
#pip install seaborn

In [26]:
#pip install tabulate

In [1]:
import pandas as pd
import numpy as np
from scipy import stats
from tabulate import tabulate

In [6]:
from scipy.stats import f
from scipy.stats import norm
from scipy.stats import t

In [2]:
data = pd.read_excel("Data_Tugas_Anreg_M13.xlsx")
data

,Y,X1,X2,X3,X4
0,78.5,7,26,6,60
1,74.3,1,29,15,52
2,104.3,11,56,8,20
3,87.6,11,31,8,47
4,95.9,7,52,6,33
5,109.2,11,55,9,22
6,102.7,3,71,17,6
7,72.5,1,31,22,44
8,93.1,2,54,18,22
9,115.9,21,47,4,26


In [8]:
datax1 = data['X1']
datax2 = data['X2']
datax3 = data['X3']
datax4 = data['X4']
datay = data['Y']
data_x = np.column_stack((datax1, datax2, datax3, datax4))

In [9]:
#Function untuk summary output regresi
def mlregression(DataX, DataY, alpha):
    banyak_baris = DataY.shape[0]
    banyak_kolom = DataX.shape[1]
    matriks_1 = []
    for i in range(banyak_baris):
        matriks_1.append(1)
    matriks_x = np.column_stack((matriks_1, DataX))
    b = np.linalg.inv(matriks_x.T @ matriks_x) @ (matriks_x.T @ DataY.T)
    
    persamaan_r = f'Y(pred) = {round(b[0],3)}'
    for i in range(banyak_kolom):
        persamaan_r += f' + {round(b[i+1],3)} X{i+1}'

    #TABEL 2 ANOVA
    ybar = np.mean(DataY)
    dfr = banyak_kolom
    dfe = banyak_baris - banyak_kolom - 1
    dft = banyak_baris - 1
    yhat = np.dot(matriks_x, b)
    e = yhat - DataY
    sse = np.sum(e**2)
    mse = sse/dfe
    ssr = np.sum((yhat-ybar)**2)
    msr = ssr/dfr
    sst = ssr + sse

    fhit = msr/mse
    pval = 1- f.cdf(fhit, dfr, dfe)

    #tabel Regression Statistics
    Rsq = ssr/sst
    Radj = 1-(1-Rsq)*(dft/dfe)
    multipleR = np.sqrt(Rsq)
    std_error = np.sqrt(mse)
    obsrvtions = banyak_baris

    #TABEL 1 REGRESSION STATISTICS
    nama_baris1 = np.array(['Multiple R', 'R Square', 'Adjusted R Square','Standard Error','Observations'])
    nama_kolom1 = np.array(['Regression Statistics'])
    kolom_hasil1 = np.array([multipleR,Rsq,Radj,std_error,obsrvtions])
    output_tabel1 = pd.DataFrame(kolom_hasil1, index=nama_baris1, columns=nama_kolom1)

    var_b = np.linalg.inv(matriks_x.T @ matriks_x) * mse
    se = np.sqrt(np.diag(var_b))

    nama_baris = np.array(['Regression', 'Residual', 'Total'])
    nama_kolom = np.array(['Sum of Squares', 'df', 'Mean Squares', 'F - Score', 'Sign. F'])
    kolom_ss = np.array([ssr, sse, sst])
    kolom_df = np.array([dfr, dfe, banyak_baris-1])
    kolom_ms = np.array([msr, mse, ""])
    kolom_f = np.array([fhit,"",""])
    kolom_p = np.array([pval,"",""])
    data_anova = np.column_stack((kolom_ss, kolom_df, kolom_ms, kolom_f, kolom_p))
    output_anova = pd.DataFrame(data_anova, index=nama_baris, columns=nama_kolom)

    #TABEL 3
    se = np.sqrt(np.diag(var_b))
    kolom_thit = []
    #loop T hitung
    for i in range(len(b)):
        persamaan = b[i]/se[i]
        kolom_thit.append(persamaan)
    #loop Confidence Interval
    t_alpha = t.ppf(1-alpha/2, dfe)
    CI_lower=[]
    CI_upper=[]
    for i in range(len(b)):
        CI_low = b[i] - t_alpha*se[i]
        CI_lower.append(CI_low)
        CI_up = b[i] + t_alpha*se[i]
        CI_upper.append(CI_up) 

    list_p_value = []
    for i in range(len(b)):
        list_p_value.append(2*(1-t.cdf(abs(kolom_thit[i]),dfe)))
    
    nama_baris3 = ["Intercept"]
    for i in range(1, len(b)):
        nama_baris3.append(f'X Variable {i}')
    nama_kolom3 = np.array(['Coefficient', 'Standard Error','T-Stat', 'P-Value', f'Lower {(1-alpha)*100}%', f'Upper {(1-alpha)*100}%'])
    tabel3_value = np.column_stack((b, se, kolom_thit, list_p_value, CI_lower, CI_upper))
    tabel3_dataframe = pd.DataFrame(tabel3_value, index = nama_baris3, columns = nama_kolom3)

    
    output_table_1 = tabulate(output_tabel1, headers='keys', tablefmt='fancy_grid', numalign="center")
    output_table_2 = tabulate(output_anova, headers='keys', tablefmt='fancy_grid',numalign="center")
    output_table_3 = tabulate(tabel3_dataframe, headers='keys', tablefmt='fancy_grid',numalign="center")
    
    print("===================================================================")
    print("\t\t\tPOSTULATED MODEL")
    print("-------------------------------------------------------------------")
    print(persamaan_r)
    print("===================================================================\n")
    print("Tabel I. Regression Statistics")
    print(output_table_1, "\n")
    print("Tabel II. ANOVA")
    print(output_table_2, "\n")
    print("Tabel III. Estimation of Parameters")
    print(output_table_3,"\n")
    Ftabel = f.ppf(1-alpha/2, dfr, dfe)
    if fhit > Ftabel:
        keputusan_overall = "Tolak H0\nTerdapat salah satu (β0,β1,β2,...,βn) yang signifikan"
    else :
        keputusan_overall = "Gagal Tolak H0\nSemua nilai koefisien tidak signifikan terhadap model"
    nama_baris_overall = np.array(['H0','H1','F-Statistic','F-Table','Keputusan'])
    nama_kolom_overall = np.array(['UJI OVERALL'])
    list_uji_overall = np.array(['β0=β1=β2=...=βn= 0', 'Minimal terdapat 1 koefisien parameter tidak sama dengan 0',fhit,Ftabel,keputusan_overall])
    df_uji_overall = pd.DataFrame(list_uji_overall, index=nama_baris_overall, columns=nama_kolom_overall)

    output_table_4 = tabulate( df_uji_overall, headers='keys', tablefmt='fancy_grid', numalign="center")
    print("Tabel IV. Overall Test")
    print(output_table_4,"\n")

    print("========================================================================================")
    print("\t\t\t\t  PARTIAL TEST")
    print("----------------------------------------------------------------------------------------")
    print("H0 : βi = 0, Variabel Xi tidak memiliki pengaruh signifikan terhadap Y (i=0,1,...,n)")
    print("H1 : βi != 0, Variabel Xi memiliki pengaruh signifikan terhadap Y (i=0,1,...,n)")
    print("========================================================================================")

    kep_tolakh0 = "Tolak H0"
    kep_gagaltolakh0 = "Gagal Tolak H0"
    list_keputusan_parsial = []
    for i in range(len(b)):
        if list_p_value[i] < alpha:
            list_keputusan_parsial.append(kep_tolakh0)
        else:
            list_keputusan_parsial.append(kep_gagaltolakh0)
    kolom_alpha = []
    for i in range(len(b)):
        kolom_alpha.append(alpha)
    nama_baris_parsial = []
    for i in range(len(b)):
        nama_baris_parsial.append(f'β{i}')
    nama_kolom_parsial = np.array(['P-Value','Alpha','Keputusan'])
    list_uji_parsial = np.column_stack((list_p_value, kolom_alpha, list_keputusan_parsial))
    df_uji_parsial = pd.DataFrame(list_uji_parsial, index=nama_baris_parsial, columns=nama_kolom_parsial)
    output_table_5 = tabulate( df_uji_parsial, headers='keys', tablefmt='fancy_grid', numalign="center")
    print("Tabel V. Partial Test Result")
    print(output_table_5)

In [10]:
mlregression(data_x, datay, 0.05)

			POSTULATED MODEL
-------------------------------------------------------------------
Y(pred) = 62.405 + 1.551 X1 + 0.51 X2 + 0.102 X3 + -0.144 X4

Tabel I. Regression Statistics
╒═══════════════════╤═════════════════════════╕
│                   │  Regression Statistics  │
╞═══════════════════╪═════════════════════════╡
│ Multiple R        │        0.991149         │
├───────────────────┼─────────────────────────┤
│ R Square          │        0.982376         │
├───────────────────┼─────────────────────────┤
│ Adjusted R Square │        0.973563         │
├───────────────────┼─────────────────────────┤
│ Standard Error    │         2.44601         │
├───────────────────┼─────────────────────────┤
│ Observations      │           13            │
╘═══════════════════╧═════════════════════════╛ 

Tabel II. ANOVA
╒════════════╤══════════════════╤══════╤═══════════════════╤════════════════════╤═══════════════════════╕
│            │  Sum of Squares  │  df  │ Mean Squares      │ F - Score 

### __Mencoba dengan data lain__

In [33]:
data2 = pd.read_excel("DATA 2 REGRESI.xlsx", sheet_name="Sheet1")
data2

,Experience,Score,Salary
0,4,78,24.0
1,7,100,43.0
2,1,86,23.7
3,5,82,34.3
4,8,86,35.8
5,10,84,38.0
6,0,75,22.2
7,1,80,23.1
8,6,83,30.0
9,6,91,33.0


In [34]:
x1 = data2['Experience']
x2 = data2['Score']
y_2 = data2['Salary']
x_gabungan = np.column_stack((x1,x2))

In [35]:
mlregression(x_gabungan, y_2, 0.05)

			POSTULATED MODEL
-------------------------------------------------------------------
Y(pred) = 3.174 + 1.404 X1 + 0.251 X2

Tabel I. Regression Statistics
╒═══════════════════╤═════════════════════════╕
│                   │  Regression Statistics  │
╞═══════════════════╪═════════════════════════╡
│ Multiple R        │        0.913334         │
├───────────────────┼─────────────────────────┤
│ R Square          │        0.834179         │
├───────────────────┼─────────────────────────┤
│ Adjusted R Square │        0.814671         │
├───────────────────┼─────────────────────────┤
│ Standard Error    │         2.41876         │
├───────────────────┼─────────────────────────┤
│ Observations      │           20            │
╘═══════════════════╧═════════════════════════╛ 

Tabel II. ANOVA
╒════════════╤══════════════════╤══════╤════════════════════╤════════════════════╤════════════════════════╕
│            │  Sum of Squares  │  df  │ Mean Squares       │ F - Score          │ Sign. F  